In [1]:
import math
import pandas as pd
import numpy as np
import wave
import os
import sys
import winsound
import pydub
import time
from pydub import AudioSegment
from pathlib import Path

In [2]:
# C = 0, C# = 1, etc.
roots = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

# 1 |-> sus 2, 2 |-> minor 3rd, 3 |-> major 3rd, 4 |-> sus 4
thirds = [1,2,3,4]

# 1 |-> flat 5th, 2 |-> perfect 5th, 3 |-> sharp 5th
fifths = [1,2,3]

# 0|-> no 7th, 1 |-> minor 7th, 2 |-> major 7th
sevenths = [0,1,2]

# 0 |-> no 9th, 1 |-> flat 9th, 2 |-> natural 9th, 3 |-> sharp 9th
ninths = [0, 1, 2, 3]

# 0 |-> no 11th, 1 |-> natural 11th, 2 |-> sharp 11th
elevenths = [0, 1, 2]

# 0 |-> no 13th, 1 |-> flat 13th, 2 |-> natural 13th
thirteenths = [0, 1, 2]

We define a chord to be a 7-tuple (i.e. an element of the direct product of the sets above) satisfying some conditions:

1. There cannot be both a minor 3rd and sharp 9, for these are the same note.

2. Similarly, there cannot be both a flat 5, and sharp 11, for these are the same note.

The size of the direct product of these sets is 5184.

There are 
-648 7-tuples failing to satify condition 1,
-864 7-tuples failing to satisfy condition 2,
-108 7-tuples that fail both 1 and 2 simultaneously.

By basic principles of combinatorics, we have that there are 5184-648-864+108=3870 different chords (in the sense that have defined them, some musicians might disagree with this definition).


In [4]:
# 7 total lists to iterate through

# chord_data = np.zeros((3456, 7))

In [42]:
# chord_data = np.zeros((5184, 7))

def cartesian_coord(*arrays):
    grid = np.meshgrid(*arrays)        
    coord_list = [entry.ravel() for entry in grid]
    points = np.vstack(coord_list).T
    points[:, [1, 0]] = points[:, [0, 1]]
    return points

# chord_data = cartesian_coord(
#     roots, 
#     thirds, 
#     fifths, 
#     sevenths, 
#     ninths, 
#     elevenths, 
#     thirteenths
# )

# chord_data=pd.DataFrame(chord_data)

# chord_data.to_csv(
#     r'E:\Academics\Jupyter Notebooks\data\Chord Data\chords dataset3.txt', 
#     header='infer', 
#     index=None, 
#     sep=' ', 
#     mode='a'
# )

In [12]:
#chord_data = cartesian_coord(roots, thirds, fifths, sevenths, ninths, elevenths, thirteenths)

In [47]:
chords = pd.read_csv(
    r'E:\Academics\Jupyter Notebooks\data\Chord Data\chords dataset.txt', 
    sep=' '
)

print(chords)
print(chords.head(10))

      root  3rd  5th  7th  9th  11th  13th                 name
0        0    1    1    0    0     0     0             Csus2 b5
1        0    1    1    0    0     0     1         Csus2 b5 b13
2        0    1    1    0    0     0     2          Csus2 b5 13
3        0    1    1    0    0     1     0          Csus2 b5 11
4        0    1    1    0    0     1     1      Csus2 b5 11 b13
...    ...  ...  ...  ...  ...   ...   ...                  ...
6943    11    4    3    1    1     0     2     Bsus4 #5 7 b9 13
6944    11    4    3    2    0     0     0        Bsus4 #5 maj7
6945    11    4    3    2    0     0     2     Bsus4 #5 maj7 13
6946    11    4    3    2    1     0     0     Bsus4 #5 maj7 b9
6947    11    4    3    2    1     0     2  Bsus4 #5 maj7 b9 13

[6948 rows x 8 columns]
   root  3rd  5th  7th  9th  11th  13th             name
0     0    1    1    0    0     0     0         Csus2 b5
1     0    1    1    0    0     0     1     Csus2 b5 b13
2     0    1    1    0    0     0   

In [4]:
def chord_name(chord, shift=0):
    # Inputs: chord; an array, shift, an integer
    # the chord array has a choice of root note, and the remaining elements
    # are the choice of extensions
    
    # Outputs: a string which is the name of the chord
    
    name = ""
    
    # Next we define a some dictionaries using our chord note conventions:
    root_dict = {
    0: 'C',
    1: 'C#',
    2: 'D',
    3: 'Eb',
    4: 'E',
    5: 'F',
    6: 'F#',
    7: 'G',
    8: 'G#',
    9: 'A',
    10: 'Bb',
    11: 'B'
    }
    
    third_dict = {
        1: 'sus2',
        2: 'm',
        3: '',
        4: 'sus4'
    }
    
    fifth_dict = {
        1: ' b5',
        2: '',
        3: ' #5'
    }
    
    seventh_dict = {
        0: '',
        1: ' 7',
        2: ' maj7'
    }
    
    ninth_dict = {
        0: '',
        1: ' b9',
        2: ' 9',
        3: ' #9'
    }
    
    eleventh_dict = {
        0: '',
        1: ' 11',
        2: ' #11'
    }
    
    thirteenth_dict = {
        0: '',
        1: ' b13',
        2: ' 13'
    }
    
    chord_array_dict = {
        0: root_dict,
        1: third_dict,
        2: fifth_dict,
        3: seventh_dict,
        4: ninth_dict,
        5: eleventh_dict,
        6: thirteenth_dict
    }
    
    for i in range(7):
        if i == 0:
            name += chord_array_dict[i][chord[i]+shift]
        else:
            name += chord_array_dict[i][chord[i]]
        
    return name

In [125]:
#names = [""]*3672
#for index, row in chords.iterrows():
    #names[index]=chord_name(row)
    
#chords['name'] = names

In [5]:
# outputs the distance of two notes, not necessarily a metric
def note_dist(a,b):
    if a < 0 or b < 0:
        return 7
    c = ((a % 12)-(b % 12)) % 12
    d = ((b % 12)-(a % 12)) % 12
    return min(c, d)

def sign(a):
    if int(a) == 0:
        return a
    else:
        return int(a/abs(a))

#print(note_dist(0, 10)) |-> 2

In [24]:
# chord |-> the elements of Z/12Z of a chord
# voicing |-> the elements of Z/12Z of a voicing of a chord
def keys_of_chord(chord):
    chord = np.array(chord)
    offset = np.array([0, chord[0]+1, chord[0]+5, chord[0]+9, chord[0]+0, chord[0]+4, chord[0]+7])
    keys = (chord+offset)%12
    del_inds = [i for i in range(3,7) if chord[i] == 0]
    keys=np.delete(keys, del_inds)
    return keys

def keys_of_voicing(voicing):
    voicing = np.array(voicing)
    keys = np.unique(voicing%12)
    return keys

# [0, 3, 7, 10, 2, 5, 8]
print(keys_of_chord([0, 1, 2, 1, 2, 1, 1]))
print(keys_of_chord([0, 1, 2, 1, 2, 0, 0]))
print(keys_of_chord([0, 1, 2, 1, 0, 1, 0]))
print(keys_of_chord([0, 1, 2, 1, 0, 0, 1]))
print(keys_of_chord([0, 1, 2, 1, 1, 2, 2]))

[ 0  2  7 10  2  5  8]
[ 0  2  7 10  2]
[ 0  2  7 10  5]
[ 0  2  7 10  8]
[ 0  2  7 10  1  6  9]


In [25]:
# chord-tuple |-> voicing of chord such that lowest note is root, remaining notes are chosen randomly
def random_initial_root_voicing(chord):
    oct_cond = 0
    
    keys = keys_of_chord(chord=chord)
    
    if keys[0] <= 3:
        oct_cond = 1
    
    l = len(keys)
    g=[1+oct_cond if n == keys[0] else 3+oct_cond if n == keys[2] else np.random.randint(3, 5) for n in keys]
    g = np.concatenate(([0+oct_cond], g))
    keys = np.concatenate(([keys[0]], keys))
    voice = 12*g+keys
    
    return voice

chord1 = [4, 1, 2, 1, 0, 1, 0]
chord2 = [2, 2, 2, 2, 0, 2, 2]
print("Chord name: ", chord_name(chord1))
print("Chord name: ", chord_name(chord2))

print("Keys of chord 1: ", keys_of_chord(chord1))
print("Keys of chord 2: ", keys_of_chord(chord2))

print("Random voicing of chord1: ", random_initial_root_voicing(chord1))
print("Keys of random voicing of chord1: ", keys_of_voicing(random_initial_root_voicing(chord1)))
print("Random voicing of chord2: ", random_initial_root_voicing(chord2))

Chord name:  Esus2 7 11
Chord name:  Dm maj7 #11 13
Keys of chord 1:  [ 4  6 11  2  9]
Keys of chord 2:  [ 2  5  9  1  8 11]
Random voicing of chord1:  [ 4 16 54 47 50 45]
Keys of random voicing of chord1:  [ 2  4  6  9 11]
Random voicing of chord2:  [14 26 53 57 49 44 59]


In [26]:
chord1 = [4, 2, 2, 1, 0, 1, 0]

def play_voicing(chord, 
                 voice=np.array([]), 
                 folder='E:\Academics\Jupyter Notebooks\data\Rhodes Notes'
                ):
    if not voice.any():
        voice = random_initial_root_voicing(chord)
    clip_folder = Path(folder)
    clips = [clip_folder / f'2RhodesNote-0{i}.wav' 
             if i < 10 else clip_folder / f'2RhodesNote-{i}.wav' 
             for i in voice]
    sounds = [AudioSegment.from_file(clip, format="wav")-20 
              for i, clip in enumerate(clips)]
    s = sounds[0]
    for i in range(len(sounds)-1):
        s=s.overlay(sounds[i+1]-2, position = i*50)
    
    combined = s+5
    
    combined.export(clip_folder / 'temp.wav', format = 'wav')
    
    winsound.PlaySound(str(clip_folder / 'temp.wav'), winsound.SND_FILENAME)
    
play_voicing(chord1, voice=random_initial_root_voicing(chord1))

In [12]:
# Replace play_voicing with this
# Returns a pydub AudioSegment of a voicing
def audio_from_voicing(chord,
                       voice=np.array([]),
                       ntuple = 4,
                       duration = 2000, 
                       tempo = 120, 
                       folder='E:\Academics\Jupyter Notebooks\data\Rhodes Notes'
                      ):
    if not voice.any():
        voice = random_initial_root_voicing(chord)
    clip_folder = Path(folder)
    clips = [clip_folder / f'2RhodesNote-0{i}.wav' 
             if i < 10 else clip_folder / f'2RhodesNote-{i}.wav' 
             for i in voice]
    sounds = [AudioSegment.from_file(clip, format="wav")-20 
              for i, clip in enumerate(clips)]
    
    #merges sound clips into one sound
    s = sounds[0]
    for i in range(len(sounds)-1):
        s = s.overlay(sounds[i+1]-2, position = i*50)
    s = s+5
    
    #converts tempo to ms/b
    #needs exception handling or something
    tempo = 60000/tempo
    if ntuple*tempo-duration <= 0.0:
        s = s[:duration]
    else:
        silence = AudioSegment.silent(duration=(ntuple*tempo-duration))
        s = s[:duration]+silence
    return s

In [11]:
def audio_from_chord_progression(chord_prog, voice_prog, ntuples, durations, tempo):
    #inputs: 
    # chord_prog, an array of names of chords
    # voice_prog, an array of voicings for each chord
    # ntuples, an array of tuplet sizes for each chord
    # durations, an array of durations for each chord
    # tempo, the desired tempo for playback
    
    # array of audio segments, summing the array gives audio for the chord progression
    chord_audio = [audio_from_voicing(chord_prog[i], voice_prog[i], ntuples[i], durations[i], tempo) 
                   for i in range(len(chord_prog))]
    s = chord_audio[0]
    for i in range(len(chord_audio)-1):
        s = s + chord_audio[i+1]
    return s

In [52]:
for i, row in chords.sample(frac=1).iterrows():
#for index, row in chords.iterrows():
    print(chord_name(row[:7]))
    chord = row[:7]
    voice = random_initial_root_voicing(chord)
    # comment the line below out to disable rootless voicings
    voice = voice[1:]
    play_voicing(chord, voice=voice)

C#sus4 b5 maj7 b13
B b5 7 #9 11
C#sus2 b5 maj7 b9 11 b13
Dm b5 maj7 b9 13
B 7 b9 11 13
G# maj7 #9 #11 b13
F 7 b9 #11
G b5 7 b9 b13
G#m #5 11 13
G #5 maj7 b9
Dm #5 b9
G b5 9
F#m b5 b9
F b5 7 9 11 13
F#sus2 #5 maj7 11 13
Bsus2 b5 7 11
F# maj7 #9 #11 b13
Dm b9 b13
C# 7 #9 b13
D #5 7 #9 11
B b5 7 b9 13
Eb 7 #9 11 13
Bbm #5 maj7 9
E b5 maj7 9
Dsus4 b5 7 b9 b13
Gm #5 7 9 11 13
Bb 7 b9 b13
Fsus4 b5 maj7 13
Am b5 maj7 b13
F#sus2 #5 maj7 b9 11 13
Csus4 7 13
Gsus4 b5 13
F# 7 #9
Fm maj7 13
Bb 7 b13
Csus2 #5 11 13
Bb b5 13
Bm b5 b9 11 b13
B b5 7 9 b13
Ebsus2 #11 13
Bm b5 11 b13
Asus2 b9 11
Gm b5 maj7 b9 13
Csus2 b5 maj7 b13
B maj7 #9 #11 b13
Bbm b5 11 b13
Bbm b5 7 b9
Dm b5 9 11 13
Bb maj7 9 11
C#sus2 #11
A b5 7 #9 11
F# 7 #9 #11 13
G#sus4 #5 maj7
Gsus4 b13
Asus2 b5 7 11 b13
Cm b5 9 13
Gm b5 maj7 b13
A b5 7 9 13
G #9 11 13
C# b5 #9 b13
Bbm b5 7 11 b13
A maj7 9
G#sus2 7 b9 11 13
Ebm maj7 11 13


KeyboardInterrupt: 

In [19]:
# Function that returns index of input chord name
def chord_finder(chord_name):
    return chords.loc[chords['name']==chord_name, chords.columns != 'name'].values.flatten()

In [20]:
# if the root is below 4, raise octave up 1
def voice_correction(voicing):
    if voicing[0] < 4:
        voicing[0]+=12
        voicing[1]+=12
    for i in range(len(voicing)):
        if voicing[i] < 27 and i > 1:
            voicing[i]+=12
        elif voicing[i] > 58:
            voicing[i]-=12
        if voicing[i] // 12 > 4:
            voicing[i] -= 12
        if voicing[i] // 12 < 2 and i > 2:
            voicing[i] += 12
    copy = sorted(voicing)
    if (copy[len(copy)-1] - copy[len(copy)-2]) > 8:
        voicing[len(voicing)-1]-=12
    return voicing

In [33]:
print(chord_finder("Asus2 7 b9 b13"))
play_voicing(chord_finder("Asus4 9"))

[9 1 2 1 1 0 1]


In [55]:
#chord_progression=['Bm 13', 'C#m 13', 'Dm 13', 'C#m 13']
#chord_progression=['Bm', 'Gm', 'Bbm', 'C#m 13']
chord_progression=['G#sus4', 'Am 7', 'C#m 13', 'Cm 13', 'F#sus4', 'G# 7', 'C 7', 'B 7']
for i in range(4):
    for i, chord in enumerate(chord_progression):
        chord1 = chord_finder(chord)
        voicing = random_initial_root_voicing(chord1)
        voicing = voice_correction(voicing)
        play_voicing(chord_finder(chord), voice = voicing)

KeyboardInterrupt: 

$\large
\textbf{Conditional Chord Voicing:} \newline \normalsize$
For a chord $c$ with voicing $v$, let $N(c, v)$ be the tuple of numerical notes on a keyboard to play that chord-voicing pair. For example, if $c=(0, 2, 2, 0, 0, 2, 0), v=(3, 4, 3, 0, 0, 4, 0)$, i.e. a Cmaj #11 chord containing notes (C, G, E, F#), then $$N(c, v)=(36, 43, 52, 54).$$

One notable problem of chord progression generation is the problem of conditional chord voicing. $\newline$
Given two chords, $c_1, c_2$, and a voicing $v_1$, how does one choose a voicing $v_2$ of $c_2$ that "works well" with $c_1, v_1$?

Note this is not easy when the two chords have a different number of notes. First we assume they have the same number of notes.

We will consider the following strategies:

a) Choose the voicing so that the movement from $c_1, v_1$ to $c_2, v_2$ is minimized. Note there could be multiple voicings that satisfy this (maybe).

b) Choose the voicing so that the high register movement is minimized

c) Fix the roots to neighboring octaves, remaining movement is minimized

d) Fix the roots to neighboring octaves, then prioritize high register movement minimzation

e) Some sort of beam search

$c_1=(0, 2, 2, 0, 0, 2, 0), v_1=(3, 4, 3, 0, 0, 4, 0), c2=(5, 2, 1, 0, 3, 0, 0)$, here $c_1, c_2$ are "C #11", and "F b5 #9" respectively.

Now, $N(c_1, v_1)=(36, 43, 52, 54), N(c_2, v_2)=(35, 45, 53, 56)$
The distance from $(c_1, v_1) \text{ to} (c_2, v_2)$ is $\sum\limits_{i=1}^4 |N(c_1, v_1)-N(c_2, v_2)|=1+2+1+2$

Algorithm 1:


In [38]:
#returns a voicing for chord2 that is compatible with voicing1
def conditional_voicing1(voicing1, chord2):
    voicing1 = np.array(voicing1)
    voicing1 = np.sort(voicing1)
    keys1 = keys_of_voicing(voicing1)
    og_keys2 = keys_of_chord(chord2)
    keys2 = np.delete(og_keys2, [0])
    # initializing what we will return
    voicing2 = -1*np.ones(len(keys2), dtype=int)
    # Setting the root notes
    voicing2[0]=keys2[0]
    voicing2[1]=keys2[0]+12

    for i in range(len(voicing1)-1):
        #select the ith highest note in chord 1
        high_note1 = voicing1[-i-1] 
        #initialize an array that will contain the note distances to the high note
        dists = np.ones(len(keys2), dtype=int)
        # fill out the note distances to the high note
        for j in range(len(dists)):
            dist = note_dist(high_note1%12, keys2[j])
            dists[j]=dist


        #index of minimum distance between note and chord
        idx = np.argmin(dists, axis=0)

        print("Step #: ", i+1 ," Note: ", high_note1, ", key: ", high_note1%12)
        print("Distances: ", dists)
        print("Key closest in chord2: ", keys2[idx], " index: ", idx)
        #now we must choose the note for this neighboring key we have identified!
        close_key = keys2[idx]
        corr_note =voicing1[-i-1]-int(math.remainder(high_note1%12-close_key, 12))
        print("Corresponding note in chord2: ", corr_note)
        voicing2[idx]=corr_note
        #keys2[idx] = -1
        
    print(voicing2)

    for i in range(len(voicing2)):
      if voicing2[i]==-1:
        high_note2 = keys2[i]
        dists = np.ones(len(np.delete(keys1, 0)), dtype=int)
        for j in range(len(dists)):
          dist = note_dist(high_note2, np.delete(voicing1, [0, 1])[j])
          dists[j]=dist
        idx = np.argmin(dists, axis=0)
        print("Keys 2: ", keys2)
        print("Voicing 2: ", voicing2)
        print("Voicing 1: ", np.delete(voicing1, [0, 1]))
        print("Keys 1: ", np.delete(keys1, 0))
        print("Distances: ", dists)
        print("Step #: ", i+1 ," Note: ", high_note2,
              ", note closest in chord1: :",
              np.delete(voicing1, [0, 1])[idx], " index: ", idx)
        direction = 1

        close_key = np.delete(keys1, 0)[idx]
        if high_note2 - close_key > note_dist(high_note, close_key):
            direction = -1
        corr_note = voicing1[idx+2]+direction*note_distance
        print("Corresponding note in chord1: ", corr_note)
        print("Close note: ", voicing1[idx+2])
        voicing2[i]=corr_note


    voicing2=np.concatenate(([og_keys2[0],og_keys2[0]+12], voicing2))
    voicing2 = voice_correction(voicing2)
    
    print("Voicing2: ", voicing2)
    print("Keys of voicing2: ", keys_of_voicing(voicing2))
    print("Keys of chord2: ", keys_of_chord(chord2))
    return voicing2

voicing1=[10, 22, 37, 41]
chord2 = chord_finder('C#m 13')

print(conditional_voicing1(voicing1, chord2))
print(keys_of_voicing(conditional_voicing1(voicing1, chord2)))
print(keys_of_chord(chord2))
play_voicing(chord2, voice=conditional_voicing1(voicing1, chord2))

Step #:  1  Note:  41 , key:  5
Distances:  [1 3 5]
Key closest in chord2:  4  index:  0
Corresponding note in chord2:  40
Step #:  2  Note:  37 , key:  1
Distances:  [3 5 3]
Key closest in chord2:  4  index:  0
Corresponding note in chord2:  40
Step #:  3  Note:  22 , key:  10
Distances:  [6 2 0]
Key closest in chord2:  10  index:  2
Corresponding note in chord2:  22
[40 16 22]
Voicing2:  [13 25 40 28 34]
Keys of voicing2:  [ 1  4 10]
Keys of chord2:  [ 1  4  8 10]
[13 25 40 28 34]
Step #:  1  Note:  41 , key:  5
Distances:  [1 3 5]
Key closest in chord2:  4  index:  0
Corresponding note in chord2:  40
Step #:  2  Note:  37 , key:  1
Distances:  [3 5 3]
Key closest in chord2:  4  index:  0
Corresponding note in chord2:  40
Step #:  3  Note:  22 , key:  10
Distances:  [6 2 0]
Key closest in chord2:  10  index:  2
Corresponding note in chord2:  22
[40 16 22]
Voicing2:  [13 25 40 28 34]
Keys of voicing2:  [ 1  4 10]
Keys of chord2:  [ 1  4  8 10]
[ 1  4 10]
[ 1  4  8 10]
Step #:  1  Note

In [39]:
def high_notes(voicing):
    voicing_highs = []
    average = 0
    for note in voicing:
        if note >= 28:
            voicing_highs.append(note)
            average += note
    average = average/len(voicing_highs)
    if average == 0:
        average = max(voicing)
    return voicing_highs, average

In [44]:
# An alternative conditional voicing function
# Needs to also return voicing with lower variance
def conditional_voicing2(voicing1, chord2):
    """
    Computes the average note in voicing1 except any bass notes
    Considers possible voicings for chord2 and chooses the one
    whose average is closest to voicing1
    
    Inputs: voicing1, the voicing of the first chord
    Inputs: chord2, the chord following the first chord that needs
    a voicing
    
    Outputs: voicing2, a voicing for chord2
    """
    voicing1_highs, average1 = high_notes(voicing1)
    voicing2 = random_initial_root_voicing(chord2)
    voicing2_highs, average2 = high_notes(voicing2)
#     for voice in all_voicings(chord2):
#         root=voice[0]%12
#         if root < 4:
#             root += 12
#         average = 0
#         for note in voice[1:]:
#             average+=note
#         average = average/(len(voice)-1)
#         if abs(average1-average) < abs(average1-average2):
#             voicing2 = np.concatenate(([root],voice[1:]))
#             average2 = average
    for voice in all_voicings(chord2):
        root=voice[0]%12
        if root < 4:
            root += 12
        
        voice_highs, average = high_notes(voice)
        if abs(average1-average) < abs(average1-average2):
            voicing2 = np.concatenate(([root],voice[1:]))
            average2 = average
    return voice_correction(voicing2)

chord1 = chord_finder('Bbm')
voicing1=np.array([10, 22, 37, 41])
chord2 = chord_finder('C#m 13')
voicing2 = conditional_voicing2(voicing1, chord2)
play_voicing(chord1, voice = voicing1)
play_voicing(chord2, voice = voicing2)

In [54]:
#chord_progression=['Bm', 'Gm', 'Bbm', 'C#m 13']
#chord_progression=['Am', 'Cm', 'Bbm', 'C#m']
#chord_progression=['Bm', 'Gm', 'Em 7', 'Gm']
chord_progression=['G#sus4', 'Am 7', 'C#m 13', 'Cm 13', 'F#sus4', 'G# 7', 'C 7', 'B 7']
n = len(chord_progression)
chord1=chord_finder(chord_progression[0])
voicing1 = random_initial_root_voicing(chord1)
voicing_prev=voicing1
for i in range(4):
    print(voicing1)
    play_voicing(chord1, voice=voicing1)
    for j in range(n-1):
        chord_prev = chord_finder(chord_progression[j])
        chord_cur = chord_finder(chord_progression[j+1])
        voicing_cur = conditional_voicing2(voicing_prev, chord_cur)
        print(voicing_cur)
        play_voicing(chord_cur, voice = voicing_cur)
        voicing_prev = voicing_cur
    voicing1 = conditional_voicing2(voicing_cur, chord1)
    #voicing1 = random_initial_root_voicing(chord1)

[ 8 20 37 39]
[ 9 36 28 43]
[13 28 32 34]
[12 27 31 33]
[ 6 35 37]
[ 8 36 27 42]
[12 28 31 46]
[11 39 30 33]
[ 8 37 15]
[ 9 24 28 31]


KeyboardInterrupt: 

In [43]:
def all_voicings(chord):
    """
    Returns a collection of reasonable rootless voicings for a chord
    """
    keys = keys_of_chord(chord)
    octaves = [2,3,4]
    n = len(keys)
    voicings = cartesian_coord(*tuple([octaves]*n))
    for i in range(len(voicings)):
        voicings[i] = 12*voicings[i]+keys
        root = keys[0]
        
    return voicings

for i, voice in enumerate(all_voicings(chord_finder("A 7 #11"))):
    if i % 5 == 0:
        print(voice)
        play_voicing([2, 2, 2, 2, 0, 2, 2], voice = voice)

[33 25 28 31 27]
[33 25 28 43 51]


KeyboardInterrupt: 

In [14]:
for i, row in chords.sample(frac=1).iterrows():
#for i, row in chords.iterrows():
    print(chord_name(row[:7]))
    chord1 = row[:7]
    chord2 = chords.iloc[i+1][:7]
    
    if i == 0:
        voicing1 = random_initial_root_voicing(chord1)
        play_voicing(row[:7])
        voicing2 = conditional_voicing1(voicing1, chord2)
        voicing1 = voicing2
    else:
        voicing2 = conditional_voicing1(voicing1, chord2)
        play_voicing(chord2, voice = voicing2)
        

F# b5 #9 13
[ 6 18  6 36 57 59]
C#m maj7 b9
[13 25 40 56 36 38 57]
Am 9 #11
[ 9 21  9 33 59 39 53]
G 7 9 #11 13
[ 7 19  7 38 53 58]
F# 9 11
[ 6 18  6 30 56 59 38]
G maj7 b9 13
[ 7 19  7 38 54 56 36]
D #9 11 13
[14 26 54 26 41 56]
G#m b5 7 b9 b13
[ 8 20  8 38 54 57 53]
D b5 maj7 #9 11
[14 26 54 56 37 53 55 58]
C# 7 #9 b13
[13 25 53 56 59 40 58]
C b5 7 #9 11 b13
[12 24 12 54 58 39 53 57]
Bb maj7 #9
[10 22 38 34 57 37 54]
E maj7 9 11 13
[ 4 16 56 28 39 54 58]
Fm 7 9 13
[ 5 17 56 29 39 55 58]
G# maj7 9 11 13
[ 8 20  8 39 55 58 38]
B #9 #11 b13
[11 23 39 54 38 53 56]
C 7 #9 11
[12 24 12 55 58 39 53 56]
Ebm 7 11
[15 27 54 27 37 56 59]
A maj7 b9 11
[ 9 21  9 40 56 58 38 53]
Em b5 b9 11 13
[ 4 16 55 28 30]
Dm b5 maj7 9 11
[14 26 53 56 37 40 55 58]
C# #9 13
[13 25 13 56 40 54]
B maj7 9 #11 13
[11 23 39 54 58 38]
A b5 7 #9 11 b13
[ 9 21  9 39 55 36 38 54]
F# b5 #9 b13
[ 6 18  6 30 57 27]
G#m b5 7 11
[ 8 20  8 38 54 37 28]
D b5 7 9 13
[14 26 54 56 36 40 55]
Am b9 #11
[ 9 21  9 33 58 39 53]
G b9 1

KeyboardInterrupt: 